In [2]:
import socket
import threading
import sys
import os

class ChatClient:

    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.server_address = (host, port)

        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.nombre_usuario = None

    def _listen_for_messages(self):
        while True:
            try:
                data = self.sock.recv(1024)
                if not data:
                    print("\n[DESCONECTADO] El servidor ha cerrado la conexión.")
                    self.sock.close()
                    os._exit(0)

                mensaje_recibido = data.decode('utf-8')

                if mensaje_recibido.startswith("Error:"):
                    print(f"\n[ERROR DEL SERVIDOR] {mensaje_recibido}")
                    self.sock.close()
                    os._exit(0)

                print(mensaje_recibido, end='')

            except Exception:
                print("\n[ERROR] Se ha perdido la conexión con el servidor.")
                self.sock.close()
                os._exit(0)

    def _send_messages_loop(self):
        try:
            while True:
                mensaje = input()

                if mensaje.lower() == 'salir':
                    break

                self.sock.sendall(mensaje.encode('utf-8'))

        except (KeyboardInterrupt, EOFError):
            print("\nSaliendo del chat...")
        finally:
            print('Cerrando conexión...')
            self.sock.close()
            sys.exit()

    def start(self):
        self.nombre_usuario = input("Ingresa tu nombre de usuario para el chat: ")
        if not self.nombre_usuario:
            print("El nombre no puede estar vacío. Saliendo.")
            return

        try:
            print(f'Conectando como "{self.nombre_usuario}" a {self.host}:{self.port}...')
            self.sock.connect(self.server_address)

            self.sock.sendall(self.nombre_usuario.encode('utf-8'))

            print(f'¡Conectado! Escribe "salir" para desconectarte.')

        except ConnectionRefusedError:
             print(f"[ERROR] No se pudo conectar. Asegúrate de que el servidor esté encendido.")
             return
        except Exception as e:
            print(f"[ERROR] No se pudo conectar: {e}")
            return

        receiver_thread = threading.Thread(target=self._listen_for_messages, daemon=True)
        receiver_thread.start()

        self._send_messages_loop()

if __name__ == "__main__":
    HOST = 'localhost'
    PORT = 8070

    cliente = ChatClient(HOST, PORT)
    cliente.start()

Conectando como "Daner" a localhost:8070...
¡Conectado! Escribe "salir" para desconectarte.

Saliendo del chat...
Cerrando conexión...


SystemExit: 

/home/salazar/anaconda3/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
